<a href="https://colab.research.google.com/github/zhiyuan-95/Amex-default-prediction-/blob/main/accessAndExpand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get data

In [ ]:
%%shell
pip --quiet install kaggle

### download data from kaggle ###

In [ ]:
from google.colab import files
# Upload the kaggle.json file
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"empanadas","key":"27d0db8ad40443c4bc6929322d9c2d3c"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
import kaggle
import subprocess
api_command = 'kaggle competitions download -c amex-default-prediction'
subprocess.run(api_command, shell = True)

### unzip to colab##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile('amex-default-prediction.zip','r') as zip_ref:
  zip_ref.extractall('/content')

### upload to google storage

In [ ]:
import google.colab
from google.cloud import storage

In [ ]:
google.colab.auth.authenticate_user()

In [ ]:
client = storage.Client()
bucket_name = 'aedp'
bucket = client.bucket(bucket_name)

file_path = '/content/train_labels.csv'
blob = bucket.blob('train_labels.csv')
blob.upload_from_filename(file_path)

# data expansion
###1. get numerical part of data and categorical part of data

###2. expand the data set

*   for the numerical part, using the describe() function in pandas to get the statistic of a customer's bank statement variables
*   for the categorical part('D_63', 'D_64', 'B_31'), also use describe in pandas to get statistic

###3. combine those two part of information, and also combine all the customers in rdd then we get a new data set

###4. those processes are run on the dataproc

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=LNsM9wTGP8xrSwn6CcK4s70ODujhEo&prompt=consent&access_type=offline&code_challenge=KvB6NiiDJS26o06HQ6Ei0eFfwnjPehxzb0D3NImB2G8&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhU7_k00WXRDfz67_KJKhMm7YzzzUe8M8eCoIRGAOUdQeDEYzW9OAY-sJu9y0UsVAg

You are now logged in as [zhiyuan.jin1201@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID

In [ ]:
!gsutil cp gs://aedp/original_data/train_data.csv /content/training_set.csv
!gsutil cp gs://aedp/original_data/train_labels.csv /content/label.csv

Copying gs://aedp/original_data/train_data.csv...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][ 15.3 GiB/ 15.3 GiB]   90.4 MiB/s                                   
Operation completed over 1 objects/15.3 GiB.                                     
Copying gs://aedp/original_data/train_labels.csv...
/ [1 files][ 29.3 MiB/ 29.3 MiB]                                                
Operation completed over 1 objects/29.3 MiB.                                     


In [ ]:
%%shell
pip --quiet install pyspark

In [ ]:
import pandas as pd
import pyspark
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
rdd_feature = sc.textFile('training_set.csv')
rdd_labels = sc.textFile('label.csv')

In [ ]:
cat= ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
col = rdd_feature.take(1)[0].split(',')
num = col[:]
for x in cat:
  num.remove(x)
num.remove('S_2')

In [ ]:
## get a smaller sample to try on local device
sample_rdd = sc.parallelize(rdd_feature.take(1000))

In [ ]:
import csv
def split(_, lines):
  if _==0:
    next(lines)
  for r in csv.reader(lines):
    yield r
sample = sample_rdd.mapPartitionsWithIndex(split)

In [ ]:
import numpy as np

In [ ]:
cat= ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
col = rdd_feature.take(1)[0].split(',')
num = col[:]
for x in cat:
  num.remove(x)
num.remove('S_2')
def split(_, lines):
  if _==0:
    next(lines)
  for r in csv.reader(lines):
    yield r
feature = sample_rdd.mapPartitionsWithIndex(split)
def preprocess_num(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu
def strToFloat(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu

def explore(rows):
  df = pd.DataFrame(rows, columns = col[1:])
  df_cat = df[cat]
  df_num = pd.DataFrame(rows, columns = col[1:]).drop(columns =cat+['S_2'])
  df_num = strToFloat(df_num)
  des_num = df_num.describe()
  num_part = list(np.array(des_num).reshape(1416))
  des_cat = df_cat.describe()
  cat_part = list(np.array(des_cat).reshape(44))
  return num_part+cat_part

expansion = feature.map(lambda x: (x[0], x[1:]))\
                  .groupByKey()\
                  .mapValues(explore)\
                  .map(lambda x: ([x[0]]+x[1]))\
                  .map(lambda x: ','.join([str(flt) for flt in x]))

In [ ]:
expansion.saveAsTextFile('testFile')

In [ ]:
labels = dict(rdd_labels.mapPartitionsWithIndex(split).collect())
add_labels = expansion.map(lambda x: x+[labels[x[0]]])\
                      .map(lambda x: (','.join([str(flt) for flt in x])))

In [ ]:
num_col = []
cat_col = []

for x in ['count', 'unique', 'top', 'freq']:
  for y in cat:
    cat_col.append(y+'_'+x)
cat_col.append('target')

for x in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']:
  for y in num[1:]:
    num_col.append(y+'_'+x)

head = '.'.join(num_col+cat_col)

In [ ]:
head = sc.parallelize([','.join(['customer_ID']+num_col+cat_col)])
output = head.union(expansion)
output.saveAsTextFile('textFile')

In [ ]:
!gcloud dataproc clusters create cluster-7b82 --enable-component-gateway --region us-east1 --zone us-east1-c --master-machine-type n1-standard-4 --master-boot-disk-size 500 --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 300 --image-version 2.0-debian10 --project bda-12345

Waiting on operation [projects/bda-12345/regions/us-central1/operations/479da88c-7179-30d5-93f6-9d5ce51ef34b].

ERROR: (gcloud.dataproc.clusters.create) Operation [projects/bda-12345/regions/us-central1/operations/479da88c-7179-30d5-93f6-9d5ce51ef34b] failed: The zone 'projects/bda-12345/zones/us-central1-c' does not have enough resources available to fulfill the request.  Try a different zone, or try again later..


In [ ]:
!gcloud config set project bda-12345
!gcloud config set compute/region us-east1
!gcloud config set compute/zone us-east1-c
!gcloud config set dataproc/region us-east1

Updated property [core/project].
Updated property [compute/region].
Updated property [compute/zone].
Updated property [dataproc/region].


In [ ]:
%%writefile expansion.py
import csv
import pandas as pd
import numpy as np
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()
rdd_feature = sc.textFile('gs://aedp/original_data/train_data.csv')
rdd_labels = sc.textFile('gs://aedp/original_data/train_labels.csv')

cat= ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
col = rdd_feature.take(1)[0].split(',')
num = col[:]
for x in cat:
  num.remove(x)
num.remove('S_2')
def split(_, lines):
  if _==0:
    next(lines)
  for r in csv.reader(lines):
    yield r
feature = rdd_feature.mapPartitionsWithIndex(split)
def preprocess_num(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu
def strToFloat(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu

def explore(rows):
  df = pd.DataFrame(rows, columns = col[1:])
  df_cat = df[cat]
  df_num = pd.DataFrame(rows, columns = col[1:]).drop(columns =cat+['S_2'])
  df_num = strToFloat(df_num)
  des_num = df_num.describe()
  num_part = list(np.array(des_num).reshape(1416))
  des_cat = df_cat.describe()
  cat_part = list(np.array(des_cat).reshape(44))
  return num_part+cat_part

expansion = feature.map(lambda x: (x[0], x[1:]))\
                  .groupByKey()\
                  .mapValues(explore)\
                  .map(lambda x: ([x[0]]+x[1]))
labels = dict(rdd_labels.mapPartitionsWithIndex(split).collect())
add_labels = expansion.map(lambda x: x+[labels[x[0]]])\
                      .map(lambda x: (','.join([str(flt) for flt in x])))
num_col = []
cat_col = []
for x in ['count', 'unique', 'top', 'freq']:
  for y in cat:
    cat_col.append(y+'_'+x)
cat_col.append('target')
for x in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']:
  for y in num[1:]:
    num_col.append(y+'_'+x)

head = sc.parallelize([','.join(['customer_ID']+num_col+cat_col)])
output = head.union(add_labels).saveAsTextFile('gs://aedp/expanded_data/expanded_data')

Overwriting expansion.py


In [ ]:
%%writefile test_expansion.py
import csv
import pandas as pd
import numpy as np
import itertools
import pyspark
from pyspark import SparkContext
sc = SparkContext()
rdd_feature = sc.textFile('gs://aedp/original_data/test_data.csv')
cat= ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
col = rdd_feature.take(1)[0].split(',')
num = col[:]
for x in cat:
  num.remove(x)
num.remove('S_2')
def split(_, lines):
  if _==0:
    next(lines)
  for r in csv.reader(lines):
    yield r
feature = rdd_feature.mapPartitionsWithIndex(split)
def preprocess_num(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu
def strToFloat(df_nu):
  df_nu = df_nu.replace('',np.nan).astype(float)
  return df_nu

def explore(rows):
  df = pd.DataFrame(rows, columns = col[1:])
  df_cat = df[cat]
  df_num = pd.DataFrame(rows, columns = col[1:]).drop(columns =cat+['S_2'])
  df_num = strToFloat(df_num)
  des_num = df_num.describe()
  num_part = list(np.array(des_num).reshape(1416))
  des_cat = df_cat.describe()
  cat_part = list(np.array(des_cat).reshape(44))
  return num_part+cat_part

expansion = feature.map(lambda x: (x[0], x[1:]))\
                  .groupByKey()\
                  .mapValues(explore)\
                  .map(lambda x: ([x[0]]+x[1]))\
                  .map(lambda x: ','.join([str(flt) for flt in x]))
num_col = []
cat_col = []
for x in ['count', 'unique', 'top', 'freq']:
  for y in cat:
    cat_col.append(y+'_'+x)
for x in ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']:
  for y in num[1:]:
    num_col.append(y+'_'+x)

head = sc.parallelize([','.join(['customer_ID']+num_col+cat_col)])
output = head.union(expansion)\
             .saveAsTextFile('gs://aedp/expanded_data/expanded_test')

Writing test_expansion.py


In [ ]:
!gcloud dataproc jobs submit pyspark --cluster cluster-7b82 test_expansion.py

Job [576f5904d0c04b10bf85344bb0416feb] submitted.
Waiting for job output...
23/07/28 02:08:47 INFO SparkEnv: Registering MapOutputTracker
23/07/28 02:08:47 INFO SparkEnv: Registering BlockManagerMaster
23/07/28 02:08:48 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/07/28 02:08:48 INFO SparkEnv: Registering OutputCommitCoordinator
23/07/28 02:08:49 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at cluster-7b82-m.us-east1-c.c.bda-12345.internal./10.142.0.10:8032
23/07/28 02:08:49 INFO AHSProxy: Connecting to Application History server at cluster-7b82-m.us-east1-c.c.bda-12345.internal./10.142.0.10:10200
23/07/28 02:08:50 INFO Configuration: resource-types.xml not found
23/07/28 02:08:50 INFO ResourceUtils: Unable to find 'resource-types.xml'.
23/07/28 02:08:51 INFO YarnClientImpl: Submitted application application_1690509826647_0001
23/07/28 02:08:52 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at cluster-7b82-m.us-east1-c.c.bd

In [ ]:
!gcloud dataproc clusters delete expansion